## Data Ingestion Stage

In [1]:
from collections import namedtuple
from dataclasses import dataclass, field
from pathlib import Path
import os
from io import BytesIO
from typing import Any
import requests

#get from constants
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils import read_yaml, create_directories

import urllib.request as request
from zipfile import ZipFile

In [2]:
DataIngestionConfig = namedtuple("DataIngestionConfig",["root_dir","source_url","local_data_file","unzip_dir"])

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: Path
    local_data_file:Path
    unzip_dir: Path

In [3]:
@dataclass
class ConfigurationManager:
    config_file_path: Path = field(default=CONFIG_FILE_PATH)
    params_file_path: Path = field(default=PARAMS_FILE_PATH)
    
    def __post_init__(self):
        self.config = read_yaml(self.config_file_path)
        self.params = read_yaml(self.params_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config
    

In [10]:
@dataclass
class DataIngestion:
    config: Any
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            
    @staticmethod
    def _get_updated_list_of_files(list_of_files:list)->list:
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]
    
    @staticmethod
    def _preprocess(zf:ZipFile, f:str, working_dir:str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f,working_dir)
            
        if os.path.getsize(target_filepath)==0:
            os.remove(target_filepath)
    
        
    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode='r') as zf:
            list_of_files= zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files=list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf=zf, f=f, working_dir=self.config.unzip_dir)
                
        
        

In [5]:

old_path = os.getcwd()

os.chdir('../')

In [6]:
new_path = os.getcwd()

In [7]:
new_path

'c:\\Users\\Tyson\\Desktop\\TysonD\\ML\\CNNClassifier\\MLOps-CNNClassifier'

In [11]:
try:
    config_=ConfigurationManager()
    data_ingestion_config = config_.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2024-09-09 08:12:39,912: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-09-09 08:12:39,919: INFO: common: created directory at: artifacts:]
[2024-09-09 08:12:39,923: INFO: common: created directory at: artifacts/data_ingestion:]
